# spine instance by unet - 3D


In [1]:
# import library
import sys

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from skimage import morphology
from skimage.io import imsave
from skimage.morphology import remove_small_objects

InteractiveShell.ast_node_interactivity = 'last_expr'
import time

from tqdm import tqdm

sys.path.append("..")
sys.path.append("..//..")
import os

import ipywidgets as ipyw
# %matplotlib inline
# %matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
#load model
import tensorflow as tf
from skimage.filters.thresholding import (threshold_isodata, threshold_li,
                                          threshold_local, threshold_mean,
                                          threshold_minimum,
                                          threshold_multiotsu,
                                          threshold_niblack, threshold_otsu,
                                          threshold_triangle, threshold_yen)

from spinelib import cflow, imgio, seg
from spinelib.cflow.blob import find_spheres, peakfilter, sphere_log
from spinelib.cflow.localthreadhold import local_threshold
from spinelib.imgio import napari_base
from spinelib.seg import segment, track
from spinelib.utils import npixel
from spinelib.utils.measure import label_statics
from utils import file_base

methods = cflow.localthreadhold.thmethods
from skimage.filters import threshold_otsu, threshold_yen
#basic fun
#-----------------------#
from spinelib.utils import measure



def show_img_info(images):
    shape=images.shape
    dtype=images.dtype
    ndim=images.ndim
    print("image shape :",shape)
    print("image dtype :",dtype)
    print("image ndim :",ndim)


def modify_masks(masks,sizeth=20):
    for n,mask in enumerate(masks):
        masks[n]=modify_mask(mask,sizeth=sizeth)
def modify_mask(mask,sizeth=20):
    denmask=mask==1
    spinemask=mask==2
    denmask=remove_small_objects(denmask,min_size=sizeth)
    spinemask=remove_small_objects(spinemask,min_size=4)
    
    #projspinemask=remove_small_objects(np.max(spinemask,axis=0),min_size=sizeth)
    # print(mask.shape[0])
    projdenmask=np.tile(np.sum(denmask,axis=0),(mask.shape[0],1,1))
    spinemask[projdenmask>3]=0
    mask=spinemask*2+denmask
    return mask
    


c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
# super params
#-----------------------#
#   paras  #
#-----------------------#

steps=2 # caculate stick gradient step lenth 1-4, larger more error , smaller than spine diameter
sigma=(2,2) # z,y,x
lambda1=1
lambda2=1.5 # lambda2 need large than lambda1
sigma_hessian = 4
sigma_supp = 100
countth=10*20 # fiber largest pixels, include z stack, that to say lenth*stackn
smallobj=200
spine_radius=35 # for iteration to growth area
maxspinesize=1000
spotrange=[3,11]
binary_func=threshold_li
use_sobel=False
search_box=[7,25,25]
keep_time=5
peak_use_gaussian=True

untmodel=None

In [3]:
# load data
#-----------------------#
#   data loader  #
#-----------------------#
filename=r"D:\data\Train\4D\origion\ctrl\20211111-24D.tif"
#-----------------------#
#   Output file path  #
#-----------------------#
ddir,shotname,suffix=file_base.split_filename(filename)
savedir=r"G:\LHJ_DATA\Train\4D\deconvolve_4D\statistic"
dendir=r"G:\LHJ_DATA\Train\4D\deconvolve_4D\remove_den"
roidir=r"G:\LHJ_DATA\Train\4D\deconvolve_4D\roi_proj"
predictdir=r"G:\LHJ_DATA\Train\4D\deconvolve_4D\predict"
areadir=os.path.join(savedir,"area")
meandir=os.path.join(savedir,"mean")
file_base.create_dir(areadir)
file_base.create_dir(meandir)
file_base.create_dir(dendir)
file_base.create_dir(roidir)
file_base.create_dir(predictdir)

areafile=os.path.join(areadir,shotname+"_area.csv")
proj_areafile=os.path.join(areadir,shotname+"_proj_area.csv")
meanfile=os.path.join(meandir,shotname+"_mean_inten.csv")
proj_meanfile=os.path.join(meandir,shotname+"_proj_meaninten.csv")
sumfile=os.path.join(meandir,shotname+"_sum_inten.csv")
proj_sumfile=os.path.join(meandir,shotname+"_proj_suminten.csv")

removedenfile=os.path.join(dendir,shotname+"_remove_den.tif")

proj_roifile=os.path.join(roidir,shotname+"_roi_proj.tif")
each_roifile=os.path.join(roidir,shotname+"_roi.tif")

maskfile=os.path.join(predictdir,shotname+"_mask.tif")
spineprfile=os.path.join(predictdir,shotname+"_spine_pr.tif")
denprfile=os.path.join(predictdir,shotname+"_den_pr.tif")
#-----------------------#
#   model loader  #
#-----------------------#
import pickle
from networks import unet

unetmodel=unet.UNet2D()
# D:\spine\segment\some_analysis_for_spine_img\models\model\unet_model\M2d_den\modelep200-loss0.010 copy.h5
# D:\spine\segment\some_analysis_for_spine_img\models\model\unet_model\M2d_den\modelep082-loss0.012 copy.h5
modelpath=r"D:/code/myspine/models/M2d_seg\modelep200-loss0.133.h5"
unetmodel.build(input_shape =(4,512,512,1))
unetmodel.load_weights(modelpath)
peakmodel=tf.keras.models.load_model(
    r"D:\code\some_analysis_for_spine_img\models\model\cnn_model", custom_objects=None, compile=True, options=None
)

imgs = imgio.ski_imread(filename)[:10]
show_img_info(imgs)

image shape : (10, 11, 512, 512)
image dtype : uint16
image ndim : 4


# PART ONE : PREDICT
track by max projection ,maybe stable


1. Let's see one img

In [9]:
#--- BINARY---#
img=imgs[0]
adth=img>binary_func(img)

#--- unet model predict spine,dendrite and background---#
mask,spineprs,denprs=unetmodel.predict_single_img(img)
projmask,_,_=unetmodel.predict_single_img(np.max(img,axis=0))
mask=modify_mask(mask,10)
spinemask=mask==2
corner=peakfilter(spineprs,5,0,use_gaussian=True,exborder=False)*spinemask
spine_label=segment.label_instance_water(spineprs,corner,spineprs>0.5,maxspinesize)
# view single frame segment
viewer=imgio.plot.imshow3d(img,[corner,spineprs,denprs],name_Is=["corner","spinepr","denpr"],label_lay=[mask,spine_label],label_lay_name=["mask","spine"])

1/1 [==============================] - 0s 343ms/step


2. run all img

In [17]:
# precit all frame
all_mask,all_spine_pr,all_den_pr= unetmodel.predict_time_imgs(imgs)
all_mask=modify_masks(all_mask,sizeth=20)
# view multi frames segment

all_spine_imgs=segment.keep_spineimg_bypr(imgs,all_mask,all_spine_pr,th=0.5)#keep_spine(imgs,all_mask)
all_adths,th,meanbg=segment.adth_func_time(imgs,binary_func)

"""illustration
imgs
"spine prob" : predict spine probability
"adth img spine" : img of spine area , masked by threshold mask and spine mask
"all segmnet" : predict classification result
"""
viewer=imgio.plot.imshow3d(imgs,
                              [all_spine_pr,all_spine_imgs*all_adths],
                              name_Is=["spine prob","adth img spine"],
                              label_lay=[all_mask],
                              label_lay_name=["all segmnet"])


Processing:   0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 383ms/step


Processing:  10%|█         | 1/10 [00:04<00:44,  4.95s/it]

1/1 [==============================] - 0s 396ms/step


Processing:  20%|██        | 2/10 [00:09<00:39,  4.92s/it]

...

1/1 [==============================] - 0s 344ms/step


Processing:  90%|█████████ | 9/10 [00:43<00:04,  4.89s/it]

1/1 [==============================] - 0s 366ms/step


Processing: 100%|██████████| 10/10 [00:49<00:00,  4.98s/it]


Exception in callback BaseAsyncIOLoop._handle_events(10936, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(10936, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

3. edit proj mask

In [18]:
#-----------------------#
#   GET projection mask label  #
#-----------------------#
structure=np.ones((3,)*3)
spine_mask_proj=morphology.binary_erosion(np.max(all_mask==2,axis=0),structure)
# ---------You can use img projection peak , just pr projection seem better by little test
# ----use img projection
# img_spine_proj=keep_spine_proj(imgs*adths,all_mask)
# img_corner=peakfilter(img_spine_proj,3,0,use_gaussian=True)*spine_mask_proj
# spine_label_img=label_instance(img_spine_proj,img_corner,spine_mask_proj)
# ----use prob projection
# pr_adths=adth_func_time(all_spine_pr,binary_func)
# pr_adths_proj=np.max(pr_adths,axis=0)
# pr_spine_proj=segment.keep_spineimg_bypr(all_spine_pr*adths,all_mask)
spine_pr_proj=np.max(all_spine_pr*all_adths,axis=0)
# find proj peak
pr_corner=peakfilter(spine_pr_proj,3,0,use_gaussian=True)*spine_mask_proj
spine_label_proj=segment.label_instance_water(spine_pr_proj,pr_corner,np.max(all_mask==2,axis=0),
                                            maxspinesize)

viewer=imgio.plot.imshow3d(imgs,
                              [spine_mask_proj,spine_pr_proj,pr_corner],
                              name_Is=["spine mask proj","spine prob proj","peak of prob"],
                              label_lay=[spine_label_proj],
                              label_lay_name=["pr label"])


Exception in callback BaseAsyncIOLoop._handle_events(10400, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(10400, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [20]:
# see and edit result
"""illustration
imgs
"pr_spine_proj" : predict spine probability' projection mask by threshold mask
"pr_corner"     : peak of pr_spine_proj
"spine_pr_proj" : spine instance segment projection
"all segmnet" : predict classification result
You need edit lable ( remove/merge some error segment,and so on)
"""
viewer=imgio.plot.imshow3d(imgs,
                              [segment.keep_spineimg_bypr(imgs,all_spine_pr,th=0.5),all_spine_pr],
                              name_Is=["spine imgs","spine prob proj"],
                              label_lay=[np.array([spine_label_proj for im in imgs])],
                              label_lay_name=["pr label"])

edit_spine_label_proj=viewer.layers["pr label"].data[0]

Mean of empty slice.
invalid value encountered in true_divide


Exception in callback BaseAsyncIOLoop._handle_events(11232, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(11232, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [21]:
# ---- resee
e_spine_label_proj=edit_spine_label_proj
# viewer=etsynseg.plot.imshow3d(keep_spine_by_prob(imgs,all_spine_pr,th=0.4),
#                               [imgs,all_spine_pr],
#                               name_Is=["img","spine prob proj"],
#                               label_lay=[np.array([spine_label_pr for im in imgs])],
#                               label_lay_name=["pr label"])
e_spine_label_proj.shape

# ----- relabel proj seg
t_edit_pr_spine_proj=edit_spine_label_proj.copy()
# edit_pr_spine_proj_t=imgio.ski_imread(proj_roifile)
t_edit_pr_spine_proj,_=segment.resortseg(t_edit_pr_spine_proj,2)

proj_spine_masks=np.array([t_edit_pr_spine_proj for im in imgs])

#proj to 3d in neuron +in spine+ in proj spine
all_spine_maskes=(all_spine_pr>0.5)*all_adths*proj_spine_masks

# spine_imgs=segment.keep_spineimg_bypr(imgs,all_mask,all_spine_pr,th=0.5,cval=meanbg)

In [22]:
viewer=imgio.plot.imshow3d(imgs,
                              [all_spine_pr,all_spine_imgs],
                              name_Is=["spine prob","spine img"],
                              label_lay=[all_spine_maskes,t_edit_pr_spine_proj],
                              label_lay_name=["spine label","spine area proj"])

Exception in callback BaseAsyncIOLoop._handle_events(11824, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(11824, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

4. save mask

In [15]:
#save roi mask
import tifffile

#edit_pr_spine_proj_t : spine max mask (same size for each frame)
imsave(proj_roifile,t_edit_pr_spine_proj.astype(np.uint16))
# imsave(maskfile,all_mask)
# imsave(spineprfile,all_spine_pr)

# all_mask : den spine back mask
tifffile.imwrite(
    maskfile,
    all_mask.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

# all_spine_pr : spine probability
tifffile.imwrite(
    spineprfile,
    all_spine_pr,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)
tifffile.imwrite(
    denprfile,
    all_den_pr,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

# nspine_maskes : spine's area mask
tifffile.imwrite(
    each_roifile,
    all_spine_maskes.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

# removedenfile : remove den area imgs
tifffile.imwrite(
    removedenfile,
    all_spine_imgs,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

D:\spine\Train\4D\deconvolve_4D\roi_proj\decon_20211119-14D_roi_proj.tif is a low contrast image


# Part Two 

1. load data

In [23]:
same_spine_maskes=imgio.ski_imread(proj_roifile)
all_mask=imgio.ski_imread(maskfile)
all_spine_pr=imgio.ski_imread(spineprfile)
all_den_pr=imgio.ski_imread(denprfile)
spine_maskes=imgio.ski_imread(each_roifile)
spine_imgs=imgio.ski_imread(removedenfile)
# spine_imgs=imgs.copy()
# spine_imgs[all_den_pr>0.4]=mean1

In [50]:
viewer=imgio.plot.imshow3d(imgs,
                              [all_spine_pr,spine_imgs],
                              name_Is=["spine prob","spine img"],
                              label_lay=[spine_maskes,t_edit_pr_spine_proj],
                              label_lay_name=["spine area label","spine area proj"])

2. caculate statistic

In [24]:
import pandas as pd
same_spine_maskes_t=np.array([same_spine_maskes])
same_spine_maskes_ts=np.array([same_spine_maskes for im in imgs])

arr=measure.label_series_statics(same_spine_maskes_t,same_spine_maskes_t,measure="area")
arr_volume_same=arr#*0.16*0.16*0.3

arr=measure.label_series_statics(imgs,spine_maskes,measure="area")
df_volume=arr*0.16*0.16*0.3
df_volume=pd.DataFrame(df_volume)

arr=measure.label_series_statics(spine_imgs,same_spine_maskes_ts,measure="intensity_mean")
backvs=[np.mean(im[ms==0]) for im,ms in zip(imgs,all_mask)]
arr[:,0]=backvs
arr=np.row_stack((arr_volume_same,arr))
df_mean=pd.DataFrame(arr)

# proj information statics
spine_maskes_proj=np.max(spine_maskes,axis=1)
spine_imgs_proj=np.max(spine_imgs,axis=1)
imgs_proj=np.max(imgs,axis=1)
all_mask_proj=np.max(all_mask,axis=1)
same_spine_maskes_t_proj=np.max(same_spine_maskes_t,axis=1)
same_spine_maskes_ts_proj=np.max(same_spine_maskes_ts,axis=1)
arr=measure.label_series_statics(imgs_proj,same_spine_maskes_t_proj,measure="area")
arr_area_same=arr#*0.16*0.16


arr=measure.label_series_statics(spine_maskes_proj,spine_maskes_proj,measure="area")
df_proj_area=arr*0.16*0.16
df_proj_area=pd.DataFrame(df_proj_area)


arr=measure.label_series_statics(spine_imgs_proj,same_spine_maskes_ts_proj,measure="intensity_mean")
backvs=[np.mean(im[ms==0]) for im,ms in zip(imgs_proj,all_mask_proj)]
arr[:,0]=backvs
arr=np.row_stack((arr_area_same,arr))
df_proj_mean=pd.DataFrame(arr)




In [25]:
# save csv result
df_volume.to_csv(areafile)
df_mean.to_csv(meanfile)
df_proj_area.to_csv(proj_areafile)
df_proj_mean.to_csv(proj_meanfile)


In [127]:
del df_volume
del df_mean
del df_proj_area
del df_proj_mean
del arr
del same_spine_maskes_ts_proj

# Part Three   

compare before deconvolve

In [ ]:

def nms(image,hitpeak,returnmask=True):
    # if clabs is None:
    #     clabs=list(np.unique(hitpeak))
    # if 0 in clabs:
    #     clabs.remove(0)
    indexs=np.argwhere(hitpeak>0)
    indexs=list(indexs)
    values=image[tuple(zip(*indexs))]
    vlabs=hitpeak[tuple(zip(*indexs))]
    zipped_pairs = zip(values,vlabs,indexs)
    #sorted_pairs = sorted(zipped_pairs,lambda x :x[2])
    zipped_pairs = list(zip(vlabs,values,indexs))
    zipped_pairs.sort(key=lambda x:(x[0],-x[1]))
    nmspoints=[]
    viewedlabs=[]
    for lab,value,ind in zipped_pairs:
        if lab in viewedlabs: continue
        else:
            viewedlabs.append(lab)
            nmspoints.append(ind)
    if returnmask:
        mask=np.zeros_like(image)
        mask[tuple(zip(*nmspoints))]=1
        return mask
    else:
        return nmspoints

def keep_lable_seg(segment,labs):
    ls=np.zeros_like(segment)
    for lab in labs:
        ls[segment==lab]=lab
    return ls


def track_by_seg(image,segment,
                 radius,num_iter,searchbox,
                 keep_time=3,resvers=None,labcounts={},
                sizeth=np.inf,method="geo",adth=None,
                smoothing=0,lambda1=1, lambda2=3):
    if segment is None:
        return
    segment=segment.copy()
    if resvers is not None:
        segment[segment==0]=resvers[segment==0]
        
    adth=local_threshold(image)
    peakmask=peakfilter(image,radius)*adth
    hitpeak=segment*peakmask
    
    prelab=set(np.unique(segment))
    curlab=set(np.unique(hitpeak))
    
    #intersec
    interseclab=prelab&curlab
    
    #nmsmask=nms(image,hitpeak,returnmask=True)*hitpeak
    #print(np.unique(nmsmask))
    intersecls=seg.segment.foreach_grow_area(image,num_iter,hitpeak,
                     searchbox=searchbox,sizeth=sizeth,
                     adth=adth,method=method,smoothing=smoothing,
                     lambda1=lambda1,lambda2=lambda2)
        
    #diff # which not find
    diflab=prelab-curlab
    nofinders=keep_lable_seg(segment,diflab)
    for lab in diflab:
        if lab not in labcounts:
            labcounts[lab]=1
        elif labcounts[lab]>keep_time:
            nofinders[nofinders==lab]=0
        else:
            labcounts[lab]+=1
    
    return intersecls,peakmask,nofinders